In [ ]:
# Required imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Embedding
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd

# Set seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Adjust TensorFlow settings for CPU optimization based on the i7-13700K capabilities
tf.config.threading.set_intra_op_parallelism_threads(16)  # Utilize all available cores
tf.config.threading.set_inter_op_parallelism_threads(8)   # Optimize parallel execution between operations

# Set paths to the CSV files (update as needed)
base_path = r'C:\Users\user\Desktop\cw\MBA\instacart-market-basket-analysis'
orders_path = f'{base_path}\\orders.csv'
order_products_prior_path = f'{base_path}\\order_products__prior.csv'
products_path = f'{base_path}\\products.csv'

# Load data from CSV files
orders = pd.read_csv(orders_path)
order_products_prior = pd.read_csv(order_products_prior_path)
products = pd.read_csv(products_path)

# Preprocessing function (modify based on dataset specifics)
def preprocess_data(orders, order_products, max_sequence_length):
    # Merge orders with prior products to get sequences per user
    merged = pd.merge(order_products, orders, on='order_id')
    # Group by user_id to create sequences of product_ids
    sequences = merged.groupby('user_id')['product_id'].apply(list).values

    # Ensure sequences are not empty, otherwise filter them out
    filtered_sequences = [seq for seq in sequences if len(seq) > 1]
    labels = [seq[-1] for seq in filtered_sequences]  # Take the last item as the label

    # Pad sequences to ensure they have the same length
    padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(filtered_sequences, maxlen=max_sequence_length, padding='post')
    return padded_sequences, np.array(labels)


# Set hyperparameters
vocab_size = products['product_id'].nunique() + 1  # Example value based on products dataset
embedding_dim = 50  # Example value
hidden_units = 100  # Example value
max_sequence_length = 200  # Example value

# Preprocess the training and testing data
X_train, y_train = preprocess_data(orders, order_products_prior, max_sequence_length)
X_test, y_test = preprocess_data(orders, order_products_prior, max_sequence_length)  # Adjust as necessary for testing data

# Function to build GRU4Rec model
def build_gru4rec(vocab_size, embedding_dim, hidden_units, max_sequence_length):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length),
        GRU(hidden_units, return_sequences=False),
        Dense(vocab_size, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Build the GRU4Rec model
gru4rec_model = build_gru4rec(vocab_size, embedding_dim, hidden_units, max_sequence_length)

# Create data generators for training and testing
batch_size = 32  # Example value

def create_generator(X, y, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.shuffle(buffer_size=1024).batch(batch_size).repeat()
    return dataset

train_generator = create_generator(X_train, y_train, batch_size)
test_generator = create_generator(X_test, y_test, batch_size)

# Determine steps per epoch
train_steps = len(X_train) // batch_size
test_steps = len(X_test) // batch_size

# Training the GRU4Rec model with reduced epochs for quicker execution
gru4rec_model.fit(train_generator,
                  steps_per_epoch=train_steps,
                  epochs=5,
                  validation_data=test_generator,
                  validation_steps=test_steps)
# Evaluation function for the GRU4Rec model
def evaluate_model_generator(model, generator, steps, k=5):
    precision, recall, hit_rate, count = 0, 0, 0, 0
    for X_batch, y_batch in generator:
        predictions = model.predict(X_batch, verbose=0)
        top_k_preds = np.argsort(predictions, axis=-1)[:, -k:]
        for i in range(len(y_batch)):
            true_item = y_batch[i]
            if true_item in top_k_preds[i]:
                precision += 1
                recall += 1
                hit_rate += 1
        count += len(y_batch)
        if count >= steps * batch_size:
            break
    precision /= count
    recall /= count
    hit_rate /= count
    return precision, recall, hit_rate

# Evaluate GRU4Rec model
gru4rec_results = evaluate_model_generator(gru4rec_model, test_generator, test_steps)
print("GRU4Rec Performance (Precision@5, Recall@5, Hit Rate):", gru4rec_results)

import csv
import os

# Save results to a CSV file
def save_results_to_csv(results, base_path, filename):
    filepath = os.path.join(base_path, filename)
    with open(filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Metric', 'Value'])
        writer.writerow(['Precision@5', results[0]])
        writer.writerow(['Recall@5', results[1]])
        writer.writerow(['Hit Rate', results[2]])
#        writer.writerow(['MRR', results[3]])

# Example usage
save_results_to_csv(gru4rec_results, base_path, 'gru4recNO1_results.csv')


Epoch 1/5


C:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


6444/6444 ━━━━━━━━━━━━━━━━━━━━ 447s 69ms/step - accuracy: 0.0116 - loss: 8.7436 - val_accuracy: 0.0845 - val_loss: 6.2161
Epoch 2/5
6444/6444 ━━━━━━━━━━━━━━━━━━━━ 437s 68ms/step - accuracy: 0.1288 - loss: 6.0016 - val_accuracy: 0.3485 - val_loss: 4.4276
Epoch 3/5
6444/6444 ━━━━━━━━━━━━━━━━━━━━ 427s 66ms/step - accuracy: 0.4366 - loss: 3.8422 - val_accuracy: 0.6527 - val_loss: 2.5299
Epoch 4/5
6444/6444 ━━━━━━━━━━━━━━━━━━━━ 420s 65ms/step - accuracy: 0.6686 - loss: 2.3601 - val_accuracy: 0.7914 - val_loss: 1.6455
Epoch 5/5
6444/6444 ━━━━━━━━━━━━━━━━━━━━ 417s 65ms/step - accuracy: 0.7904 - loss: 1.5374 - val_accuracy: 0.8593 - val_loss: 1.1731
GRU4Rec Performance (Precision@5, Recall@5, Hit Rate): (0.89754034761018, 0.89754034761018, 0.89754034761018)


In [ ]:
# Required imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Embedding
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Set seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Adjust TensorFlow settings for CPU optimization based on the i7-13700K capabilities
tf.config.threading.set_intra_op_parallelism_threads(16)  # Utilize all available cores
tf.config.threading.set_inter_op_parallelism_threads(8)   # Optimize parallel execution between operations

# Set paths to the CSV files (update as needed)
base_path = r'C:\Users\user\Desktop\cw\MBA\instacart-market-basket-analysis'
orders_path = f'{base_path}\\orders.csv'
order_products_prior_path = f'{base_path}\\order_products__prior.csv'
products_path = f'{base_path}\\products.csv'

# Load data from CSV files
orders = pd.read_csv(orders_path)
order_products_prior = pd.read_csv(order_products_prior_path)
products = pd.read_csv(products_path)

# Preprocessing function (modify based on dataset specifics)
def preprocess_data(orders, order_products, max_sequence_length):
    # Merge orders with prior products to get sequences per user
    merged = pd.merge(order_products, orders, on='order_id')
    # Group by user_id to create sequences of product_ids
    sequences = merged.groupby('user_id')['product_id'].apply(list).values

    # Ensure sequences are not empty, otherwise filter them out
    filtered_sequences = [seq for seq in sequences if len(seq) > 1]
    labels = [seq[-1] for seq in filtered_sequences]  # Take the last item as the label

    # Pad sequences to ensure they have the same length
    padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(filtered_sequences, maxlen=max_sequence_length, padding='post')
    return padded_sequences, np.array(labels)

# Set hyperparameters
vocab_size = products['product_id'].nunique() + 1  # Example value based on products dataset
embedding_dim = 100  # Adjusted for better representation
hidden_units = 128  # Adjusted hidden units
max_sequence_length = 100  # Example value

# Preprocess the data
X, y = preprocess_data(orders, order_products_prior, max_sequence_length)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to build GRU4Rec model
def build_gru4rec(vocab_size, embedding_dim, hidden_units, max_sequence_length):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=embedding_dim),
        GRU(hidden_units, return_sequences=False),
        Dense(vocab_size, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Build the GRU4Rec model
gru4rec_model = build_gru4rec(vocab_size, embedding_dim, hidden_units, max_sequence_length)

# Create data generators for training and testing
batch_size = 32  # Example value

def create_generator(X, y, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.shuffle(buffer_size=1024).batch(batch_size).repeat()
    return dataset

train_generator = create_generator(X_train, y_train, batch_size)
test_generator = create_generator(X_test, y_test, batch_size)

# Determine steps per epoch
train_steps = len(X_train) // batch_size
test_steps = len(X_test) // batch_size

# Training the GRU4Rec model with more epochs for better learning
gru4rec_model.fit(train_generator,
                  steps_per_epoch=train_steps,
                  epochs=10,  # Increased number of epochs
                  validation_data=test_generator,
                  validation_steps=test_steps)

# Evaluation function for the GRU4Rec model with additional metrics
def evaluate_model_generator(model, generator, steps, k=5):
    precision, recall, hit_rate, mrr, count = 0, 0, 0, 0, 0
    for X_batch, y_batch in generator:
        predictions = model.predict(X_batch, verbose=0)
        top_k_preds = np.argsort(predictions, axis=-1)[:, -k:]
        for i in range(len(y_batch)):
            true_item = y_batch[i]
            rank = np.where(top_k_preds[i] == true_item)[0]
            if len(rank) > 0:
                rank = rank[0] + 1
                precision += 1
                recall += 1
                hit_rate += 1
                mrr += 1 / rank
        count += len(y_batch)
        if count >= steps * batch_size:
            break
    precision /= count
    recall /= count
    hit_rate /= count
    mrr /= count
    return precision, recall, hit_rate, mrr

# Evaluate GRU4Rec model
gru4rec_results = evaluate_model_generator(gru4rec_model, test_generator, test_steps)
print("GRU4Rec Performance (Precision@5, Recall@5, Hit Rate, MRR):", gru4rec_results)

import csv
import os

# Save results to a CSV file
def save_results_to_csv(results, base_path, filename):
    filepath = os.path.join(base_path, filename)
    with open(filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Metric', 'Value'])
        writer.writerow(['Precision@5', results[0]])
        writer.writerow(['Recall@5', results[1]])
        writer.writerow(['Hit Rate', results[2]])
        writer.writerow(['MRR', results[3]])

# Example usage
save_results_to_csv(gru4rec_results, base_path, 'gru4rec_resultsO1.csv')


Epoch 1/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 307s 59ms/step - accuracy: 0.0230 - loss: 8.4617 - val_accuracy: 0.2189 - val_loss: 5.6836
Epoch 2/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 313s 61ms/step - accuracy: 0.3461 - loss: 4.5928 - val_accuracy: 0.6794 - val_loss: 2.9802
Epoch 3/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 316s 61ms/step - accuracy: 0.7414 - loss: 2.0019 - val_accuracy: 0.8152 - val_loss: 2.0455
Epoch 4/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 318s 62ms/step - accuracy: 0.8749 - loss: 0.9765 - val_accuracy: 0.8603 - val_loss: 1.7018
Epoch 5/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 318s 62ms/step - accuracy: 0.9393 - loss: 0.5294 - val_accuracy: 0.8843 - val_loss: 1.5302
Epoch 6/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 312s 60ms/step - accuracy: 0.9694 - loss: 0.3008 - val_accuracy: 0.8936 - val_loss: 1.4197
Epoch 7/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 313s 61ms/step - accuracy: 0.9815 - loss: 0.1924 - val_accuracy: 0.9001 - val_loss: 1.3432
Epoch 8/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 313s 61ms/step - accuracy: 

In [ ]:
# Required imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Embedding
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import csv
import os

# Set seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Adjust TensorFlow settings for CPU optimization
tf.config.threading.set_intra_op_parallelism_threads(16)  # Utilize all available cores
tf.config.threading.set_inter_op_parallelism_threads(8)   # Optimize parallel execution between operations

# Set paths to the CSV files (update as needed)
base_path = r'C:\Users\user\Desktop\cw\MBA\instacart-market-basket-analysis'
orders_path = f'{base_path}\\orders.csv'
order_products_prior_path = f'{base_path}\\order_products__prior.csv'
products_path = f'{base_path}\\products.csv'

# Load data from CSV files
orders = pd.read_csv(orders_path)
order_products_prior = pd.read_csv(order_products_prior_path)
products = pd.read_csv(products_path)

# Preprocessing function
def preprocess_data(orders, order_products, max_sequence_length):
    merged = pd.merge(order_products, orders, on='order_id')
    sequences = merged.groupby('user_id')['product_id'].apply(list).values
    filtered_sequences = [seq for seq in sequences if len(seq) > 1]
    labels = [seq[-1] for seq in filtered_sequences]
    padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(filtered_sequences, maxlen=max_sequence_length, padding='post')
    return padded_sequences, np.array(labels)

# Set hyperparameters
vocab_size = products['product_id'].nunique() + 1
embedding_dim = 100
hidden_units = 250
max_sequence_length = 20
batch_size = 16
epochs = 10
learning_rate = 0.0001

# Preprocess the data
X, y = preprocess_data(orders, order_products_prior, max_sequence_length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to build GRU4Rec model
def build_gru4rec(vocab_size, embedding_dim, hidden_units, max_sequence_length):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=embedding_dim),
        GRU(hidden_units, return_sequences=False, dropout=0.2),
        Dense(vocab_size, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Build the GRU4Rec model
gru4rec_model = build_gru4rec(vocab_size, embedding_dim, hidden_units, max_sequence_length)

# Create data generators
def create_generator(X, y, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    return dataset.shuffle(buffer_size=1024).batch(batch_size).repeat()

train_generator = create_generator(X_train, y_train, batch_size)
test_generator = create_generator(X_test, y_test, batch_size)
train_steps = len(X_train) // batch_size
test_steps = len(X_test) // batch_size

# Train the model
gru4rec_model.fit(train_generator,
                  steps_per_epoch=train_steps,
                  epochs=epochs,
                  validation_data=test_generator,
                  validation_steps=test_steps)

# Evaluation function
def evaluate_model_generator(model, generator, steps, k=10):
    precision, recall, hit_rate, mrr, count = 0, 0, 0, 0, 0
    for X_batch, y_batch in generator:
        predictions = model.predict(X_batch, verbose=0)
        top_k_preds = np.argsort(predictions, axis=-1)[:, -k:]
        for i in range(len(y_batch)):
            true_item = y_batch[i]
            rank = np.where(top_k_preds[i] == true_item)[0]
            if len(rank) > 0:
                rank = rank[0] + 1
                precision += 1
                recall += 1
                hit_rate += 1
                mrr += 1 / rank
        count += len(y_batch)
        if count >= steps * batch_size:
            break
    precision /= count
    recall /= count
    hit_rate /= count
    mrr /= count
    return precision, recall, hit_rate, mrr

# Evaluate and save results
gru4rec_results = evaluate_model_generator(gru4rec_model, test_generator, test_steps)
print("GRU4Rec Performance (Precision@10, Recall@10, Hit Rate, MRR):", gru4rec_results)

def save_results_to_csv(results, base_path, filename, hyperparameters):
    filepath = os.path.join(base_path, filename)
    with open(filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Metric', 'Value'])
        writer.writerow(['Precision@10', results[0]])
        writer.writerow(['Recall@10', results[1]])
        writer.writerow(['Hit Rate', results[2]])
        writer.writerow(['MRR', results[3]])
        writer.writerow([])
        writer.writerow(['Hyperparameters'])
        for key, value in hyperparameters.items():
            writer.writerow([key, value])

# Save the results with hyperparameters
hyperparameters = {
    'embedding_dim': embedding_dim,
    'hidden_units': hidden_units,
    'max_sequence_length': max_sequence_length,
    'batch_size': batch_size,
    'epochs': epochs,
    'learning_rate': learning_rate
}
save_results_to_csv(gru4rec_results, base_path, 'gru4rec_results_with_hyperparameters1.csv', hyperparameters)


Epoch 1/10
10310/10310 ━━━━━━━━━━━━━━━━━━━━ 696s 67ms/step - accuracy: 0.0097 - loss: 9.1440 - val_accuracy: 0.0122 - val_loss: 8.0438
Epoch 2/10
10310/10310 ━━━━━━━━━━━━━━━━━━━━ 702s 68ms/step - accuracy: 0.0191 - loss: 7.6132 - val_accuracy: 0.0453 - val_loss: 7.5382
Epoch 3/10
10310/10310 ━━━━━━━━━━━━━━━━━━━━ 714s 69ms/step - accuracy: 0.0660 - loss: 6.8836 - val_accuracy: 0.1345 - val_loss: 6.8614
Epoch 4/10
10310/10310 ━━━━━━━━━━━━━━━━━━━━ 709s 69ms/step - accuracy: 0.1546 - loss: 6.0701 - val_accuracy: 0.2368 - val_loss: 6.2011
Epoch 5/10
10310/10310 ━━━━━━━━━━━━━━━━━━━━ 706s 69ms/step - accuracy: 0.2598 - loss: 5.2979 - val_accuracy: 0.3404 - val_loss: 5.5809
Epoch 6/10
10310/10310 ━━━━━━━━━━━━━━━━━━━━ 698s 68ms/step - accuracy: 0.3729 - loss: 4.5826 - val_accuracy: 0.4333 - val_loss: 5.0183
Epoch 7/10
10310/10310 ━━━━━━━━━━━━━━━━━━━━ 706s 69ms/step - accuracy: 0.4751 - loss: 3.9377 - val_accuracy: 0.5122 - val_loss: 4.4984
Epoch 8/10
10310/10310 ━━━━━━━━━━━━━━━━━━━━ 707s 69ms/s

In [ ]:
# Required imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Embedding
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import csv
import os

# Set seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Adjust TensorFlow settings for CPU optimization
tf.config.threading.set_intra_op_parallelism_threads(16)  # Utilize all available cores
tf.config.threading.set_inter_op_parallelism_threads(8)   # Optimize parallel execution between operations

# Set paths to the CSV files (update as needed)
base_path = r'C:\Users\user\Desktop\cw\MBA\instacart-market-basket-analysis'
orders_path = f'{base_path}\\orders.csv'
order_products_prior_path = f'{base_path}\\order_products__prior.csv'
products_path = f'{base_path}\\products.csv'

# Load data from CSV files
orders = pd.read_csv(orders_path)
order_products_prior = pd.read_csv(order_products_prior_path)
products = pd.read_csv(products_path)

# Preprocessing function
def preprocess_data(orders, order_products, max_sequence_length):
    merged = pd.merge(order_products, orders, on='order_id')
    sequences = merged.groupby('user_id')['product_id'].apply(list).values
    filtered_sequences = [seq for seq in sequences if len(seq) > 1]
    labels = [seq[-1] for seq in filtered_sequences]
    padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(filtered_sequences, maxlen=max_sequence_length, padding='post')
    return padded_sequences, np.array(labels)

# Set hyperparameters
vocab_size = products['product_id'].nunique() + 1
embedding_dim = 100
hidden_units = 200
max_sequence_length = 20
batch_size = 32
epochs = 10
learning_rate = 0.0001

# Preprocess the data
X, y = preprocess_data(orders, order_products_prior, max_sequence_length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to build GRU4Rec model
def build_gru4rec(vocab_size, embedding_dim, hidden_units, max_sequence_length):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=embedding_dim),
        GRU(hidden_units, return_sequences=False, dropout=0.2),
        Dense(vocab_size, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Build the GRU4Rec model
gru4rec_model = build_gru4rec(vocab_size, embedding_dim, hidden_units, max_sequence_length)

# Create data generators
def create_generator(X, y, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    return dataset.shuffle(buffer_size=1024).batch(batch_size).repeat()

train_generator = create_generator(X_train, y_train, batch_size)
test_generator = create_generator(X_test, y_test, batch_size)
train_steps = len(X_train) // batch_size
test_steps = len(X_test) // batch_size

# Train the model
gru4rec_model.fit(train_generator,
                  steps_per_epoch=train_steps,
                  epochs=epochs,
                  validation_data=test_generator,
                  validation_steps=test_steps)

# Evaluation function
def evaluate_model_generator(model, generator, steps, k=10):
    precision, recall, hit_rate, mrr, count = 0, 0, 0, 0, 0
    for X_batch, y_batch in generator:
        predictions = model.predict(X_batch, verbose=0)
        top_k_preds = np.argsort(predictions, axis=-1)[:, -k:]
        for i in range(len(y_batch)):
            true_item = y_batch[i]
            rank = np.where(top_k_preds[i] == true_item)[0]
            if len(rank) > 0:
                rank = rank[0] + 1
                precision += 1
                recall += 1
                hit_rate += 1
                mrr += 1 / rank
        count += len(y_batch)
        if count >= steps * batch_size:
            break
    precision /= count
    recall /= count
    hit_rate /= count
    mrr /= count
    return precision, recall, hit_rate, mrr

# Evaluate and save results
gru4rec_results = evaluate_model_generator(gru4rec_model, test_generator, test_steps)
print("GRU4Rec Performance (Precision@10, Recall@10, Hit Rate, MRR):", gru4rec_results)

def save_results_to_csv(results, base_path, filename, hyperparameters):
    filepath = os.path.join(base_path, filename)
    with open(filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Metric', 'Value'])
        writer.writerow(['Precision@10', results[0]])
        writer.writerow(['Recall@10', results[1]])
        writer.writerow(['Hit Rate', results[2]])
        writer.writerow(['MRR', results[3]])
        writer.writerow([])
        writer.writerow(['Hyperparameters'])
        for key, value in hyperparameters.items():
            writer.writerow([key, value])

# Save the results with hyperparameters
hyperparameters = {
    'embedding_dim': embedding_dim,
    'hidden_units': hidden_units,
    'max_sequence_length': max_sequence_length,
    'batch_size': batch_size,
    'epochs': epochs,
    'learning_rate': learning_rate
}
save_results_to_csv(gru4rec_results, base_path, 'gru4rec_results_with_hyperparameters2.csv', hyperparameters)


Epoch 1/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 416s 80ms/step - accuracy: 0.0109 - loss: 9.3020 - val_accuracy: 0.0084 - val_loss: 8.2223
Epoch 2/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 416s 81ms/step - accuracy: 0.0119 - loss: 7.8640 - val_accuracy: 0.0165 - val_loss: 7.9627
Epoch 3/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 410s 80ms/step - accuracy: 0.0223 - loss: 7.4855 - val_accuracy: 0.0321 - val_loss: 7.7477
Epoch 4/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 420s 81ms/step - accuracy: 0.0386 - loss: 7.1260 - val_accuracy: 0.0565 - val_loss: 7.5416
Epoch 5/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 421s 82ms/step - accuracy: 0.0645 - loss: 6.7931 - val_accuracy: 0.1000 - val_loss: 7.2929
Epoch 6/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 421s 82ms/step - accuracy: 0.1100 - loss: 6.4156 - val_accuracy: 0.1474 - val_loss: 6.9530
Epoch 7/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 421s 82ms/step - accuracy: 0.1597 - loss: 6.0055 - val_accuracy: 0.1949 - val_loss: 6.6131
Epoch 8/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 422s 82ms/step - accuracy: 

In [ ]:
# Required imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Embedding
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import csv
import os

# Set seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Adjust TensorFlow settings for CPU optimization
tf.config.threading.set_intra_op_parallelism_threads(16)  # Utilize all available cores
tf.config.threading.set_inter_op_parallelism_threads(8)   # Optimize parallel execution between operations

# Set paths to the CSV files (update as needed)
base_path = r'C:\Users\user\Desktop\cw\MBA\instacart-market-basket-analysis'
orders_path = f'{base_path}\\orders.csv'
order_products_prior_path = f'{base_path}\\order_products__prior.csv'
products_path = f'{base_path}\\products.csv'

# Load data from CSV files
orders = pd.read_csv(orders_path)
order_products_prior = pd.read_csv(order_products_prior_path)
products = pd.read_csv(products_path)

# Preprocessing function
def preprocess_data(orders, order_products, max_sequence_length):
    merged = pd.merge(order_products, orders, on='order_id')
    sequences = merged.groupby('user_id')['product_id'].apply(list).values
    filtered_sequences = [seq for seq in sequences if len(seq) > 1]
    labels = [seq[-1] for seq in filtered_sequences]
    padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(filtered_sequences, maxlen=max_sequence_length, padding='post')
    return padded_sequences, np.array(labels)

# Set hyperparameters
vocab_size = products['product_id'].nunique() + 1
embedding_dim = 100
hidden_units = 200
max_sequence_length = 20
batch_size = 32
epochs = 10
learning_rate = 0.0005

# Preprocess the data
X, y = preprocess_data(orders, order_products_prior, max_sequence_length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to build GRU4Rec model
def build_gru4rec(vocab_size, embedding_dim, hidden_units, max_sequence_length):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=embedding_dim),
        GRU(hidden_units, return_sequences=False, dropout=0.2),
        Dense(vocab_size, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Build the GRU4Rec model
gru4rec_model = build_gru4rec(vocab_size, embedding_dim, hidden_units, max_sequence_length)

# Create data generators
def create_generator(X, y, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    return dataset.shuffle(buffer_size=1024).batch(batch_size).repeat()

train_generator = create_generator(X_train, y_train, batch_size)
test_generator = create_generator(X_test, y_test, batch_size)
train_steps = len(X_train) // batch_size
test_steps = len(X_test) // batch_size

# Train the model
gru4rec_model.fit(train_generator,
                  steps_per_epoch=train_steps,
                  epochs=epochs,
                  validation_data=test_generator,
                  validation_steps=test_steps)

# Evaluation function
def evaluate_model_generator(model, generator, steps, k=10):
    precision, recall, hit_rate, mrr, count = 0, 0, 0, 0, 0
    for X_batch, y_batch in generator:
        predictions = model.predict(X_batch, verbose=0)
        top_k_preds = np.argsort(predictions, axis=-1)[:, -k:]
        for i in range(len(y_batch)):
            true_item = y_batch[i]
            rank = np.where(top_k_preds[i] == true_item)[0]
            if len(rank) > 0:
                rank = rank[0] + 1
                precision += 1
                recall += 1
                hit_rate += 1
                mrr += 1 / rank
        count += len(y_batch)
        if count >= steps * batch_size:
            break
    precision /= count
    recall /= count
    hit_rate /= count
    mrr /= count
    return precision, recall, hit_rate, mrr

# Evaluate and save results
gru4rec_results = evaluate_model_generator(gru4rec_model, test_generator, test_steps)
print("GRU4Rec Performance (Precision@10, Recall@10, Hit Rate, MRR):", gru4rec_results)

def save_results_to_csv(results, base_path, filename, hyperparameters):
    filepath = os.path.join(base_path, filename)
    with open(filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Metric', 'Value'])
        writer.writerow(['Precision@10', results[0]])
        writer.writerow(['Recall@10', results[1]])
        writer.writerow(['Hit Rate', results[2]])
        writer.writerow(['MRR', results[3]])
        writer.writerow([])
        writer.writerow(['Hyperparameters'])
        for key, value in hyperparameters.items():
            writer.writerow([key, value])

# Save the results with hyperparameters
hyperparameters = {
    'embedding_dim': embedding_dim,
    'hidden_units': hidden_units,
    'max_sequence_length': max_sequence_length,
    'batch_size': batch_size,
    'epochs': epochs,
    'learning_rate': learning_rate
}
save_results_to_csv(gru4rec_results, base_path, 'gru4rec_results_with_hyperparameters3.csv', hyperparameters)


Epoch 1/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 433s 84ms/step - accuracy: 0.0268 - loss: 8.4870 - val_accuracy: 0.2386 - val_loss: 5.8008
Epoch 2/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 428s 83ms/step - accuracy: 0.3336 - loss: 4.8940 - val_accuracy: 0.6249 - val_loss: 3.3515
Epoch 3/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 428s 83ms/step - accuracy: 0.6823 - loss: 2.4929 - val_accuracy: 0.7841 - val_loss: 2.2517
Epoch 4/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 428s 83ms/step - accuracy: 0.8423 - loss: 1.3064 - val_accuracy: 0.8473 - val_loss: 1.8066
Epoch 5/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 426s 83ms/step - accuracy: 0.9250 - loss: 0.7264 - val_accuracy: 0.8753 - val_loss: 1.5701
Epoch 6/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 430s 83ms/step - accuracy: 0.9714 - loss: 0.4075 - val_accuracy: 0.8914 - val_loss: 1.4244
Epoch 7/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 427s 83ms/step - accuracy: 0.9918 - loss: 0.2258 - val_accuracy: 0.9023 - val_loss: 1.3417
Epoch 8/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 428s 83ms/step - accuracy: 

In [ ]:
# Required imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Embedding
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import csv
import os

# Set seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Adjust TensorFlow settings for CPU optimization
tf.config.threading.set_intra_op_parallelism_threads(16)  # Utilize all available cores
tf.config.threading.set_inter_op_parallelism_threads(8)   # Optimize parallel execution between operations

# Set paths to the CSV files (update as needed)
base_path = r'C:\Users\user\Desktop\cw\MBA\instacart-market-basket-analysis'
orders_path = f'{base_path}\\orders.csv'
order_products_prior_path = f'{base_path}\\order_products__prior.csv'
products_path = f'{base_path}\\products.csv'

# Load data from CSV files
orders = pd.read_csv(orders_path)
order_products_prior = pd.read_csv(order_products_prior_path)
products = pd.read_csv(products_path)

# Preprocessing function
def preprocess_data(orders, order_products, max_sequence_length):
    merged = pd.merge(order_products, orders, on='order_id')
    sequences = merged.groupby('user_id')['product_id'].apply(list).values
    filtered_sequences = [seq for seq in sequences if len(seq) > 1]
    labels = [seq[-1] for seq in filtered_sequences]
    padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(filtered_sequences, maxlen=max_sequence_length, padding='post')
    return padded_sequences, np.array(labels)

# Set hyperparameters
vocab_size = products['product_id'].nunique() + 1
embedding_dim = 100
hidden_units = 300
max_sequence_length = 20
batch_size = 32
epochs = 10
learning_rate = 0.0003

# Preprocess the data
X, y = preprocess_data(orders, order_products_prior, max_sequence_length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to build GRU4Rec model
def build_gru4rec(vocab_size, embedding_dim, hidden_units, max_sequence_length):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=embedding_dim),
        GRU(hidden_units, return_sequences=False, dropout=0.2),
        Dense(vocab_size, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Build the GRU4Rec model
gru4rec_model = build_gru4rec(vocab_size, embedding_dim, hidden_units, max_sequence_length)

# Create data generators
def create_generator(X, y, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    return dataset.shuffle(buffer_size=1024).batch(batch_size).repeat()

train_generator = create_generator(X_train, y_train, batch_size)
test_generator = create_generator(X_test, y_test, batch_size)
train_steps = len(X_train) // batch_size
test_steps = len(X_test) // batch_size

# Train the model
gru4rec_model.fit(train_generator,
                  steps_per_epoch=train_steps,
                  epochs=epochs,
                  validation_data=test_generator,
                  validation_steps=test_steps)

# Evaluation function
def evaluate_model_generator(model, generator, steps, k=10):
    precision, recall, hit_rate, mrr, count = 0, 0, 0, 0, 0
    for X_batch, y_batch in generator:
        predictions = model.predict(X_batch, verbose=0)
        top_k_preds = np.argsort(predictions, axis=-1)[:, -k:]
        for i in range(len(y_batch)):
            true_item = y_batch[i]
            rank = np.where(top_k_preds[i] == true_item)[0]
            if len(rank) > 0:
                rank = rank[0] + 1
                precision += 1
                recall += 1
                hit_rate += 1
                mrr += 1 / rank
        count += len(y_batch)
        if count >= steps * batch_size:
            break
    precision /= count
    recall /= count
    hit_rate /= count
    mrr /= count
    return precision, recall, hit_rate, mrr

# Evaluate and save results
gru4rec_results = evaluate_model_generator(gru4rec_model, test_generator, test_steps)
print("GRU4Rec Performance (Precision@10, Recall@10, Hit Rate, MRR):", gru4rec_results)

def save_results_to_csv(results, base_path, filename, hyperparameters):
    filepath = os.path.join(base_path, filename)
    with open(filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Metric', 'Value'])
        writer.writerow(['Precision@10', results[0]])
        writer.writerow(['Recall@10', results[1]])
        writer.writerow(['Hit Rate', results[2]])
        writer.writerow(['MRR', results[3]])
        writer.writerow([])
        writer.writerow(['Hyperparameters'])
        for key, value in hyperparameters.items():
            writer.writerow([key, value])

# Save the results with hyperparameters
hyperparameters = {
    'embedding_dim': embedding_dim,
    'hidden_units': hidden_units,
    'max_sequence_length': max_sequence_length,
    'batch_size': batch_size,
    'epochs': epochs,
    'learning_rate': learning_rate
}
save_results_to_csv(gru4rec_results, base_path, 'gru4rec_results_with_hyperparameters4.csv', hyperparameters)


Epoch 1/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 611s 118ms/step - accuracy: 0.0157 - loss: 8.7723 - val_accuracy: 0.0996 - val_loss: 6.7609
Epoch 2/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 601s 117ms/step - accuracy: 0.1613 - loss: 6.0260 - val_accuracy: 0.3911 - val_loss: 4.9332
Epoch 3/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 602s 117ms/step - accuracy: 0.4499 - loss: 3.9472 - val_accuracy: 0.6112 - val_loss: 3.5356
Epoch 4/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 603s 117ms/step - accuracy: 0.6715 - loss: 2.4293 - val_accuracy: 0.7256 - val_loss: 2.7443
Epoch 5/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 602s 117ms/step - accuracy: 0.8149 - loss: 1.4928 - val_accuracy: 0.7879 - val_loss: 2.2996
Epoch 6/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 600s 116ms/step - accuracy: 0.9102 - loss: 0.9112 - val_accuracy: 0.8223 - val_loss: 2.0238
Epoch 7/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 598s 116ms/step - accuracy: 0.9654 - loss: 0.5496 - val_accuracy: 0.8435 - val_loss: 1.8509
Epoch 8/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 589s 114ms/step - ac

In [ ]:
# Required imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Embedding
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import csv
import os

# Set seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Adjust TensorFlow settings for CPU optimization
tf.config.threading.set_intra_op_parallelism_threads(16)  # Utilize all available cores
tf.config.threading.set_inter_op_parallelism_threads(8)   # Optimize parallel execution between operations

# Set paths to the CSV files (update as needed)
base_path = r'C:\Users\user\Desktop\cw\MBA\instacart-market-basket-analysis'
orders_path = f'{base_path}\\orders.csv'
order_products_prior_path = f'{base_path}\\order_products__prior.csv'
products_path = f'{base_path}\\products.csv'

# Load data from CSV files
orders = pd.read_csv(orders_path)
order_products_prior = pd.read_csv(order_products_prior_path)
products = pd.read_csv(products_path)

# Preprocessing function
def preprocess_data(orders, order_products, max_sequence_length):
    merged = pd.merge(order_products, orders, on='order_id')
    sequences = merged.groupby('user_id')['product_id'].apply(list).values
    filtered_sequences = [seq for seq in sequences if len(seq) > 1]
    labels = [seq[-1] for seq in filtered_sequences]
    padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(filtered_sequences, maxlen=max_sequence_length, padding='post')
    return padded_sequences, np.array(labels)

# Set hyperparameters
vocab_size = products['product_id'].nunique() + 1
embedding_dim = 100
hidden_units = 250
max_sequence_length = 20
batch_size = 16
epochs = 20
learning_rate = 0.0001

# Preprocess the data
X, y = preprocess_data(orders, order_products_prior, max_sequence_length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to build GRU4Rec model
def build_gru4rec(vocab_size, embedding_dim, hidden_units, max_sequence_length):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=embedding_dim),
        GRU(hidden_units, return_sequences=False, dropout=0.2),
        Dense(vocab_size, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Build the GRU4Rec model
gru4rec_model = build_gru4rec(vocab_size, embedding_dim, hidden_units, max_sequence_length)

# Create data generators
def create_generator(X, y, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    return dataset.shuffle(buffer_size=1024).batch(batch_size).repeat()

train_generator = create_generator(X_train, y_train, batch_size)
test_generator = create_generator(X_test, y_test, batch_size)
train_steps = len(X_train) // batch_size
test_steps = len(X_test) // batch_size

# Train the model
gru4rec_model.fit(train_generator,
                  steps_per_epoch=train_steps,
                  epochs=epochs,
                  validation_data=test_generator,
                  validation_steps=test_steps)

# Evaluation function
def evaluate_model_generator(model, generator, steps, k=10):
    precision, recall, hit_rate, mrr, count = 0, 0, 0, 0, 0
    for X_batch, y_batch in generator:
        predictions = model.predict(X_batch, verbose=0)
        top_k_preds = np.argsort(predictions, axis=-1)[:, -k:]
        for i in range(len(y_batch)):
            true_item = y_batch[i]
            rank = np.where(top_k_preds[i] == true_item)[0]
            if len(rank) > 0:
                rank = rank[0] + 1
                precision += 1
                recall += 1
                hit_rate += 1
                mrr += 1 / rank
        count += len(y_batch)
        if count >= steps * batch_size:
            break
    precision /= count
    recall /= count
    hit_rate /= count
    mrr /= count
    return precision, recall, hit_rate, mrr

# Evaluate and save results
gru4rec_results = evaluate_model_generator(gru4rec_model, test_generator, test_steps)
print("GRU4Rec Performance (Precision@10, Recall@10, Hit Rate, MRR):", gru4rec_results)

def save_results_to_csv(results, base_path, filename, hyperparameters):
    filepath = os.path.join(base_path, filename)
    with open(filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Metric', 'Value'])
        writer.writerow(['Precision@10', results[0]])
        writer.writerow(['Recall@10', results[1]])
        writer.writerow(['Hit Rate', results[2]])
        writer.writerow(['MRR', results[3]])
        writer.writerow([])
        writer.writerow(['Hyperparameters'])
        for key, value in hyperparameters.items():
            writer.writerow([key, value])

# Save the results with hyperparameters
hyperparameters = {
    'embedding_dim': embedding_dim,
    'hidden_units': hidden_units,
    'max_sequence_length': max_sequence_length,
    'batch_size': batch_size,
    'epochs': epochs,
    'learning_rate': learning_rate
}
save_results_to_csv(gru4rec_results, base_path, 'gru4rec_results_with_hyperparameters5.csv', hyperparameters)


Epoch 1/20
10310/10310 ━━━━━━━━━━━━━━━━━━━━ 926s 90ms/step - accuracy: 0.0107 - loss: 9.1493 - val_accuracy: 0.0169 - val_loss: 8.0542
Epoch 2/20
10310/10310 ━━━━━━━━━━━━━━━━━━━━ 916s 89ms/step - accuracy: 0.0256 - loss: 7.6086 - val_accuracy: 0.0522 - val_loss: 7.5204
Epoch 3/20
10310/10310 ━━━━━━━━━━━━━━━━━━━━ 918s 89ms/step - accuracy: 0.0699 - loss: 6.8674 - val_accuracy: 0.1395 - val_loss: 6.8591
Epoch 4/20
10310/10310 ━━━━━━━━━━━━━━━━━━━━ 918s 89ms/step - accuracy: 0.1580 - loss: 6.0497 - val_accuracy: 0.2348 - val_loss: 6.1979
Epoch 5/20
10310/10310 ━━━━━━━━━━━━━━━━━━━━ 919s 89ms/step - accuracy: 0.2606 - loss: 5.2825 - val_accuracy: 0.3427 - val_loss: 5.5973
Epoch 6/20
10310/10310 ━━━━━━━━━━━━━━━━━━━━ 916s 89ms/step - accuracy: 0.3751 - loss: 4.5735 - val_accuracy: 0.4371 - val_loss: 5.0281
Epoch 7/20
10310/10310 ━━━━━━━━━━━━━━━━━━━━ 916s 89ms/step - accuracy: 0.4813 - loss: 3.9110 - val_accuracy: 0.5159 - val_loss: 4.4780
Epoch 8/20
10310/10310 ━━━━━━━━━━━━━━━━━━━━ 913s 89ms/s

In [ ]:
# Required imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Embedding
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import csv
import os

# Set seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Adjust TensorFlow settings for CPU optimization
tf.config.threading.set_intra_op_parallelism_threads(16)  # Utilize all available cores
tf.config.threading.set_inter_op_parallelism_threads(8)   # Optimize parallel execution between operations

# Set paths to the CSV files (update as needed)
base_path = r'C:\Users\user\Desktop\cw\MBA\instacart-market-basket-analysis'
orders_path = f'{base_path}\\orders.csv'
order_products_prior_path = f'{base_path}\\order_products__prior.csv'
products_path = f'{base_path}\\products.csv'

# Load data from CSV files
orders = pd.read_csv(orders_path)
order_products_prior = pd.read_csv(order_products_prior_path)
products = pd.read_csv(products_path)

# Preprocessing function
def preprocess_data(orders, order_products, max_sequence_length):
    merged = pd.merge(order_products, orders, on='order_id')
    sequences = merged.groupby('user_id')['product_id'].apply(list).values
    filtered_sequences = [seq for seq in sequences if len(seq) > 1]
    labels = [seq[-1] for seq in filtered_sequences]
    padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(filtered_sequences, maxlen=max_sequence_length, padding='post')
    return padded_sequences, np.array(labels)

# Set hyperparameters
vocab_size = products['product_id'].nunique() + 1
embedding_dim = 200
hidden_units = 400
max_sequence_length = 20
batch_size = 32
epochs = 10
learning_rate = 0.0007

# Preprocess the data
X, y = preprocess_data(orders, order_products_prior, max_sequence_length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Function to build GRU4Rec model
def build_gru4rec(vocab_size, embedding_dim, hidden_units, max_sequence_length):
    model = Sequential([
        Embedding(input_dim=vocab_size, output_dim=embedding_dim),
        GRU(hidden_units, return_sequences=False, dropout=0.2),
        Dense(vocab_size, activation='softmax')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Build the GRU4Rec model
gru4rec_model = build_gru4rec(vocab_size, embedding_dim, hidden_units, max_sequence_length)

# Create data generators
def create_generator(X, y, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    return dataset.shuffle(buffer_size=1024).batch(batch_size).repeat()

train_generator = create_generator(X_train, y_train, batch_size)
test_generator = create_generator(X_test, y_test, batch_size)
train_steps = len(X_train) // batch_size
test_steps = len(X_test) // batch_size

# Train the model
gru4rec_model.fit(train_generator,
                  steps_per_epoch=train_steps,
                  epochs=epochs,
                  validation_data=test_generator,
                  validation_steps=test_steps)

# Evaluation function
def evaluate_model_generator(model, generator, steps, k=10):
    precision, recall, hit_rate, mrr, count = 0, 0, 0, 0, 0
    for X_batch, y_batch in generator:
        predictions = model.predict(X_batch, verbose=0)
        top_k_preds = np.argsort(predictions, axis=-1)[:, -k:]
        for i in range(len(y_batch)):
            true_item = y_batch[i]
            rank = np.where(top_k_preds[i] == true_item)[0]
            if len(rank) > 0:
                rank = rank[0] + 1
                precision += 1
                recall += 1
                hit_rate += 1
                mrr += 1 / rank
        count += len(y_batch)
        if count >= steps * batch_size:
            break
    precision /= count
    recall /= count
    hit_rate /= count
    mrr /= count
    return precision, recall, hit_rate, mrr

# Evaluate and save results
gru4rec_results = evaluate_model_generator(gru4rec_model, test_generator, test_steps)
print("GRU4Rec Performance (Precision@10, Recall@10, Hit Rate, MRR):", gru4rec_results)

def save_results_to_csv(results, base_path, filename, hyperparameters):
    filepath = os.path.join(base_path, filename)
    with open(filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Metric', 'Value'])
        writer.writerow(['Precision@10', results[0]])
        writer.writerow(['Recall@10', results[1]])
        writer.writerow(['Hit Rate', results[2]])
        writer.writerow(['MRR', results[3]])
        writer.writerow([])
        writer.writerow(['Hyperparameters'])
        for key, value in hyperparameters.items():
            writer.writerow([key, value])

# Save the results with hyperparameters
hyperparameters = {
    'embedding_dim': embedding_dim,
    'hidden_units': hidden_units,
    'max_sequence_length': max_sequence_length,
    'batch_size': batch_size,
    'epochs': epochs,
    'learning_rate': learning_rate
}
save_results_to_csv(gru4rec_results, base_path, 'gru4rec_results_with_hyperparameters6.csv', hyperparameters)


Epoch 1/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 681s 132ms/step - accuracy: 0.1449 - loss: 7.5790 - val_accuracy: 0.7227 - val_loss: 2.5255
Epoch 2/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 630s 122ms/step - accuracy: 0.7770 - loss: 1.8486 - val_accuracy: 0.8639 - val_loss: 1.5499
Epoch 3/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 607s 118ms/step - accuracy: 0.9187 - loss: 0.6352 - val_accuracy: 0.8978 - val_loss: 1.2813
Epoch 4/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 623s 121ms/step - accuracy: 0.9807 - loss: 0.2051 - val_accuracy: 0.9130 - val_loss: 1.1594
Epoch 5/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 675s 131ms/step - accuracy: 0.9999 - loss: 0.0381 - val_accuracy: 0.9264 - val_loss: 1.0740
Epoch 6/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 675s 131ms/step - accuracy: 1.0000 - loss: 0.0030 - val_accuracy: 0.9319 - val_loss: 1.0346
Epoch 7/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 646s 125ms/step - accuracy: 1.0000 - loss: 8.4789e-04 - val_accuracy: 0.9360 - val_loss: 1.0015
Epoch 8/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 629s 122ms/step 

In [ ]:
# Required imports
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LayerNormalization, Dropout, MultiHeadAttention, Add
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import csv
import os

# Set seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Adjust TensorFlow settings for CPU optimization
tf.config.threading.set_intra_op_parallelism_threads(16)
tf.config.threading.set_inter_op_parallelism_threads(8)

# Set paths to the CSV files
base_path = r'C:\Users\user\Desktop\cw\MBA\instacart-market-basket-analysis'
orders_path = f'{base_path}\\orders.csv'
order_products_prior_path = f'{base_path}\\order_products__prior.csv'
products_path = f'{base_path}\\products.csv'

# Load data from CSV files
orders = pd.read_csv(orders_path, usecols=['order_id', 'user_id'])
order_products_prior = pd.read_csv(order_products_prior_path, usecols=['order_id', 'product_id'])
products = pd.read_csv(products_path, usecols=['product_id', 'product_name'])

# Preprocessing function
def preprocess_data(orders, order_products, max_sequence_length):
    merged = pd.merge(order_products, orders, on='order_id')
    sequences = merged.groupby('user_id')['product_id'].apply(list).values
    filtered_sequences = [seq for seq in sequences if len(seq) > 1]
    labels = [seq[-1] for seq in filtered_sequences]
    padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(filtered_sequences, maxlen=max_sequence_length, padding='post')
    return padded_sequences, np.array(labels)

# Set hyperparameters to match Script 1
vocab_size = products['product_id'].nunique() + 1
embedding_dim = 100
max_sequence_length = 100  # Reduced length
num_heads = 4
ff_dim = 128  # Match hidden units
num_layers = 2  # Reduced layers to match Script 1
dropout_rate = 0.1

# Preprocess the data
X, y = preprocess_data(orders, order_products_prior, max_sequence_length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Positional encoding function
def positional_encoding(maxlen, d_model):
    pos = np.arange(maxlen)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    pos_encoding = pos * angle_rates
    pos_encoding[:, 0::2] = np.sin(pos_encoding[:, 0::2])
    pos_encoding[:, 1::2] = np.cos(pos_encoding[:, 1::2])
    return tf.convert_to_tensor(pos_encoding, dtype=tf.float32)

# Transformer Encoder Layer
def transformer_encoder(inputs, num_heads, ff_dim, dropout_rate):
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(inputs, inputs)
    attention_output = Dropout(dropout_rate)(attention_output)
    attention_output = Add()([inputs, attention_output])
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)

    # Feed-forward layer
    ff_output = Dense(ff_dim, activation='relu')(attention_output)
    ff_output = Dense(inputs.shape[-1])(ff_output)
    ff_output = Dropout(dropout_rate)(ff_output)
    ff_output = Add()([attention_output, ff_output])
    return LayerNormalization(epsilon=1e-6)(ff_output)

# Build BERT4Rec model
def build_bert4rec(vocab_size, max_sequence_length, embedding_dim, num_heads, ff_dim, num_layers, dropout_rate):
    inputs = Input(shape=(max_sequence_length,))
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, mask_zero=True)(inputs)

    # Add positional encoding
    pos_encoding = positional_encoding(max_sequence_length, embedding_dim)
    embeddings = embedding_layer + pos_encoding

    # Stack Transformer encoder layers
    x = embeddings
    for _ in range(num_layers):
        x = transformer_encoder(x, num_heads, ff_dim, dropout_rate)

    # Output layer for prediction
    output = Dense(vocab_size, activation='softmax')(x[:, -1, :])

    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Build and compile the model
bert4rec_model = build_bert4rec(vocab_size, max_sequence_length, embedding_dim, num_heads, ff_dim, num_layers, dropout_rate)

# Create data generators for training and testing
batch_size = 32

def create_generator(X, y, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.shuffle(buffer_size=1024).batch(batch_size).repeat()
    return dataset

train_generator = create_generator(X_train, y_train, batch_size)
test_generator = create_generator(X_test, y_test, batch_size)

# Determine steps per epoch
train_steps = len(X_train) // batch_size
test_steps = len(X_test) // batch_size

# Train the BERT4Rec model with early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
bert4rec_model.fit(train_generator,
                   steps_per_epoch=train_steps,
                   epochs=10,
                   validation_data=test_generator,
                   validation_steps=test_steps,
                   callbacks=[early_stopping])

# Evaluation function
def evaluate_model_generator(model, generator, steps, k=5):
    precision, recall, hit_rate, mrr, count = 0, 0, 0, 0, 0
    generator_iterator = iter(generator)  # Create an iterator from the dataset
    for step in range(steps):
        X_batch, y_batch = next(generator_iterator)  # Use the iterator
        predictions = model.predict(X_batch, verbose=0)
        top_k_preds = np.argsort(predictions, axis=-1)[:, -k:]
        for i in range(len(y_batch)):
            true_item = y_batch[i]
            rank = np.where(top_k_preds[i] == true_item)[0]
            if len(rank) > 0:
                rank = rank[0] + 1
                precision += 1
                recall += 1
                hit_rate += 1
                mrr += 1 / rank
        count += len(y_batch)
    precision /= count
    recall /= count
    hit_rate /= count
    mrr /= count
    return precision, recall, hit_rate, mrr

# Evaluate the BERT4Rec model
bert4rec_results = evaluate_model_generator(bert4rec_model, test_generator, test_steps)
print("BERT4Rec Performance (Precision@5, Recall@5, Hit Rate, MRR):", bert4rec_results)

# Save results to a CSV file
def save_results_to_csv(results, base_path, filename):
    filepath = os.path.join(base_path, filename)
    with open(filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Metric', 'Value'])
        writer.writerow(['Precision@5', results[0]])
        writer.writerow(['Recall@5', results[1]])
        writer.writerow(['Hit Rate', results[2]])
        writer.writerow(['MRR', results[3]])

# Example usage
save_results_to_csv(bert4rec_results, base_path, 'bert4rec_results.csv')


Epoch 1/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 792s 153ms/step - accuracy: 0.0086 - loss: 9.2163 - val_accuracy: 0.0123 - val_loss: 8.3871
Epoch 2/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 761s 148ms/step - accuracy: 0.0136 - loss: 7.7728 - val_accuracy: 0.0263 - val_loss: 8.2231
Epoch 3/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 777s 151ms/step - accuracy: 0.0266 - loss: 7.3841 - val_accuracy: 0.0435 - val_loss: 7.9927
Epoch 4/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 861s 167ms/step - accuracy: 0.0464 - loss: 6.9653 - val_accuracy: 0.0817 - val_loss: 7.7525
Epoch 5/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 883s 171ms/step - accuracy: 0.0772 - loss: 6.5650 - val_accuracy: 0.0940 - val_loss: 7.5958
Epoch 6/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 1039s 202ms/step - accuracy: 0.1042 - loss: 6.1800 - val_accuracy: 0.1285 - val_loss: 7.3568
Epoch 7/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 967s 188ms/step - accuracy: 0.1352 - loss: 5.7830 - val_accuracy: 0.1813 - val_loss: 7.0109
Epoch 8/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 1054s 205ms/step - 

In [ ]:
# Required imports
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LayerNormalization, Dropout, MultiHeadAttention, Add
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import csv
import os

# Set seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Adjust TensorFlow settings for CPU optimization
tf.config.threading.set_intra_op_parallelism_threads(16)
tf.config.threading.set_inter_op_parallelism_threads(8)

# Set paths to the CSV files
base_path = r'C:\Users\user\Desktop\cw\MBA\instacart-market-basket-analysis'
orders_path = f'{base_path}\\orders.csv'
order_products_prior_path = f'{base_path}\\order_products__prior.csv'
products_path = f'{base_path}\\products.csv'

# Load data from CSV files
orders = pd.read_csv(orders_path, usecols=['order_id', 'user_id'])
order_products_prior = pd.read_csv(order_products_prior_path, usecols=['order_id', 'product_id'])
products = pd.read_csv(products_path, usecols=['product_id', 'product_name'])

# Preprocessing function
def preprocess_data(orders, order_products, max_sequence_length):
    merged = pd.merge(order_products, orders, on='order_id')
    sequences = merged.groupby('user_id')['product_id'].apply(list).values
    filtered_sequences = [seq for seq in sequences if len(seq) > 1]
    labels = [seq[-1] for seq in filtered_sequences]
    padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(filtered_sequences, maxlen=max_sequence_length, padding='post')
    return padded_sequences, np.array(labels)

# Set hyperparameters to match Script 1
vocab_size = products['product_id'].nunique() + 1
embedding_dim = 150
max_sequence_length = 100  # Reduced length
num_heads = 4
ff_dim = 128  # Match hidden units
num_layers = 2  # Reduced layers to match Script 1
dropout_rate = 0.2

# Preprocess the data
X, y = preprocess_data(orders, order_products_prior, max_sequence_length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Positional encoding function
def positional_encoding(maxlen, d_model):
    pos = np.arange(maxlen)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    pos_encoding = pos * angle_rates
    pos_encoding[:, 0::2] = np.sin(pos_encoding[:, 0::2])
    pos_encoding[:, 1::2] = np.cos(pos_encoding[:, 1::2])
    return tf.convert_to_tensor(pos_encoding, dtype=tf.float32)

# Transformer Encoder Layer
def transformer_encoder(inputs, num_heads, ff_dim, dropout_rate):
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(inputs, inputs)
    attention_output = Dropout(dropout_rate)(attention_output)
    attention_output = Add()([inputs, attention_output])
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)

    # Feed-forward layer
    ff_output = Dense(ff_dim, activation='relu')(attention_output)
    ff_output = Dense(inputs.shape[-1])(ff_output)
    ff_output = Dropout(dropout_rate)(ff_output)
    ff_output = Add()([attention_output, ff_output])
    return LayerNormalization(epsilon=1e-6)(ff_output)

# Build BERT4Rec model
def build_bert4rec(vocab_size, max_sequence_length, embedding_dim, num_heads, ff_dim, num_layers, dropout_rate):
    inputs = Input(shape=(max_sequence_length,))
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, mask_zero=True)(inputs)

    # Add positional encoding
    pos_encoding = positional_encoding(max_sequence_length, embedding_dim)
    embeddings = embedding_layer + pos_encoding

    # Stack Transformer encoder layers
    x = embeddings
    for _ in range(num_layers):
        x = transformer_encoder(x, num_heads, ff_dim, dropout_rate)

    # Output layer for prediction
    output = Dense(vocab_size, activation='softmax')(x[:, -1, :])

    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Build and compile the model
bert4rec_model = build_bert4rec(vocab_size, max_sequence_length, embedding_dim, num_heads, ff_dim, num_layers, dropout_rate)

# Create data generators for training and testing
batch_size = 32

def create_generator(X, y, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.shuffle(buffer_size=1024).batch(batch_size).repeat()
    return dataset

train_generator = create_generator(X_train, y_train, batch_size)
test_generator = create_generator(X_test, y_test, batch_size)

# Determine steps per epoch
train_steps = len(X_train) // batch_size
test_steps = len(X_test) // batch_size

# Train the BERT4Rec model with early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
bert4rec_model.fit(train_generator,
                   steps_per_epoch=train_steps,
                   epochs=10,
                   validation_data=test_generator,
                   validation_steps=test_steps,
                   callbacks=[early_stopping])

# Evaluation function
def evaluate_model_generator(model, generator, steps, k=5):
    precision, recall, hit_rate, mrr, count = 0, 0, 0, 0, 0
    generator_iterator = iter(generator)  # Create an iterator from the dataset
    for step in range(steps):
        X_batch, y_batch = next(generator_iterator)  # Use the iterator
        predictions = model.predict(X_batch, verbose=0)
        top_k_preds = np.argsort(predictions, axis=-1)[:, -k:]
        for i in range(len(y_batch)):
            true_item = y_batch[i]
            rank = np.where(top_k_preds[i] == true_item)[0]
            if len(rank) > 0:
                rank = rank[0] + 1
                precision += 1
                recall += 1
                hit_rate += 1
                mrr += 1 / rank
        count += len(y_batch)
    precision /= count
    recall /= count
    hit_rate /= count
    mrr /= count
    return precision, recall, hit_rate, mrr

# Evaluate the BERT4Rec model
bert4rec_results = evaluate_model_generator(bert4rec_model, test_generator, test_steps)
print("BERT4Rec Performance (Precision@5, Recall@5, Hit Rate, MRR):", bert4rec_results)

# Save results to a CSV file
def save_results_to_csv(results, base_path, filename):
    filepath = os.path.join(base_path, filename)
    with open(filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Metric', 'Value'])
        writer.writerow(['Precision@5', results[0]])
        writer.writerow(['Recall@5', results[1]])
        writer.writerow(['Hit Rate', results[2]])
        writer.writerow(['MRR', results[3]])

# Example usage
save_results_to_csv(bert4rec_results, base_path, 'bert4rec_results2.csv')


Epoch 1/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 910s 176ms/step - accuracy: 0.0088 - loss: 9.2702 - val_accuracy: 0.0175 - val_loss: 8.3247
Epoch 2/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 919s 178ms/step - accuracy: 0.0174 - loss: 7.7134 - val_accuracy: 0.0260 - val_loss: 8.0866
Epoch 3/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 997s 193ms/step - accuracy: 0.0271 - loss: 7.3413 - val_accuracy: 0.0400 - val_loss: 8.0138
Epoch 4/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 1083s 210ms/step - accuracy: 0.0482 - loss: 7.0065 - val_accuracy: 0.0541 - val_loss: 7.9080
Epoch 5/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 1080s 209ms/step - accuracy: 0.0590 - loss: 6.8192 - val_accuracy: 0.0528 - val_loss: 7.9273
Epoch 6/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 1084s 210ms/step - accuracy: 0.0666 - loss: 6.6956 - val_accuracy: 0.0533 - val_loss: 7.8810
Epoch 7/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 1082s 210ms/step - accuracy: 0.0789 - loss: 6.5704 - val_accuracy: 0.0633 - val_loss: 7.8549
Epoch 8/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 1085s 210ms/step

In [ ]:
# Required imports
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LayerNormalization, Dropout, MultiHeadAttention, Add
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import csv
import os

# Set seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Adjust TensorFlow settings for CPU optimization
tf.config.threading.set_intra_op_parallelism_threads(16)
tf.config.threading.set_inter_op_parallelism_threads(8)

# Set paths to the CSV files
base_path = r'C:\Users\user\Desktop\cw\MBA\instacart-market-basket-analysis'
orders_path = f'{base_path}\\orders.csv'
order_products_prior_path = f'{base_path}\\order_products__prior.csv'
products_path = f'{base_path}\\products.csv'

# Load data from CSV files
orders = pd.read_csv(orders_path, usecols=['order_id', 'user_id'])
order_products_prior = pd.read_csv(order_products_prior_path, usecols=['order_id', 'product_id'])
products = pd.read_csv(products_path, usecols=['product_id', 'product_name'])

# Preprocessing function
def preprocess_data(orders, order_products, max_sequence_length):
    merged = pd.merge(order_products, orders, on='order_id')
    sequences = merged.groupby('user_id')['product_id'].apply(list).values
    filtered_sequences = [seq for seq in sequences if len(seq) > 1]
    labels = [seq[-1] for seq in filtered_sequences]
    padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(filtered_sequences, maxlen=max_sequence_length, padding='post')
    return padded_sequences, np.array(labels)

# Set hyperparameters to match Script 1
vocab_size = products['product_id'].nunique() + 1
embedding_dim = 100
max_sequence_length = 50
num_heads = 4
ff_dim = 128
num_layers = 2
dropout_rate = 0.1

# Preprocess the data
X, y = preprocess_data(orders, order_products_prior, max_sequence_length)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Positional encoding function
def positional_encoding(maxlen, d_model):
    pos = np.arange(maxlen)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    pos_encoding = pos * angle_rates
    pos_encoding[:, 0::2] = np.sin(pos_encoding[:, 0::2])
    pos_encoding[:, 1::2] = np.cos(pos_encoding[:, 1::2])
    return tf.convert_to_tensor(pos_encoding, dtype=tf.float32)

# Transformer Encoder Layer
def transformer_encoder(inputs, num_heads, ff_dim, dropout_rate):
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(inputs, inputs)
    attention_output = Dropout(dropout_rate)(attention_output)
    attention_output = Add()([inputs, attention_output])
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)

    # Feed-forward layer
    ff_output = Dense(ff_dim, activation='relu')(attention_output)
    ff_output = Dense(inputs.shape[-1])(ff_output)
    ff_output = Dropout(dropout_rate)(ff_output)
    ff_output = Add()([attention_output, ff_output])
    return LayerNormalization(epsilon=1e-6)(ff_output)

# Build BERT4Rec model
def build_bert4rec(vocab_size, max_sequence_length, embedding_dim, num_heads, ff_dim, num_layers, dropout_rate):
    inputs = Input(shape=(max_sequence_length,))
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, mask_zero=True)(inputs)

    # Add positional encoding
    pos_encoding = positional_encoding(max_sequence_length, embedding_dim)
    embeddings = embedding_layer + pos_encoding

    # Stack Transformer encoder layers
    x = embeddings
    for _ in range(num_layers):
        x = transformer_encoder(x, num_heads, ff_dim, dropout_rate)

    # Output layer for prediction
    output = Dense(vocab_size, activation='softmax')(x[:, -1, :])

    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Build and compile the model
bert4rec_model = build_bert4rec(vocab_size, max_sequence_length, embedding_dim, num_heads, ff_dim, num_layers, dropout_rate)

# Create data generators for training and testing
batch_size = 64

def create_generator(X, y, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.shuffle(buffer_size=1024).batch(batch_size).repeat()
    return dataset

train_generator = create_generator(X_train, y_train, batch_size)
test_generator = create_generator(X_test, y_test, batch_size)

# Determine steps per epoch
train_steps = len(X_train) // batch_size
test_steps = len(X_test) // batch_size

# Train the BERT4Rec model with early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
bert4rec_model.fit(train_generator,
                   steps_per_epoch=train_steps,
                   epochs=10,
                   validation_data=test_generator,
                   validation_steps=test_steps,
                   callbacks=[early_stopping])

# Evaluation function
def evaluate_model_generator(model, generator, steps, k=5):
    precision, recall, hit_rate, mrr, count = 0, 0, 0, 0, 0
    generator_iterator = iter(generator)  # Create an iterator from the dataset
    for step in range(steps):
        X_batch, y_batch = next(generator_iterator)  # Use the iterator
        predictions = model.predict(X_batch, verbose=0)
        top_k_preds = np.argsort(predictions, axis=-1)[:, -k:]
        for i in range(len(y_batch)):
            true_item = y_batch[i]
            rank = np.where(top_k_preds[i] == true_item)[0]
            if len(rank) > 0:
                rank = rank[0] + 1
                precision += 1
                recall += 1
                hit_rate += 1
                mrr += 1 / rank
        count += len(y_batch)
    precision /= count
    recall /= count
    hit_rate /= count
    mrr /= count
    return precision, recall, hit_rate, mrr

# Evaluate the BERT4Rec model
bert4rec_results = evaluate_model_generator(bert4rec_model, test_generator, test_steps)
print("BERT4Rec Performance (Precision@5, Recall@5, Hit Rate, MRR):", bert4rec_results)

# Save results to a CSV file
def save_results_to_csv(results, base_path, filename):
    filepath = os.path.join(base_path, filename)
    with open(filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Metric', 'Value'])
        writer.writerow(['Precision@5', results[0]])
        writer.writerow(['Recall@5', results[1]])
        writer.writerow(['Hit Rate', results[2]])
        writer.writerow(['MRR', results[3]])

# Example usage
save_results_to_csv(bert4rec_results, base_path, 'bert4rec_results3.csv')


Epoch 1/10
2577/2577 ━━━━━━━━━━━━━━━━━━━━ 392s 151ms/step - accuracy: 0.0198 - loss: 8.7210 - val_accuracy: 0.1260 - val_loss: 5.9951
Epoch 2/10
2577/2577 ━━━━━━━━━━━━━━━━━━━━ 383s 149ms/step - accuracy: 0.1930 - loss: 5.1033 - val_accuracy: 0.3402 - val_loss: 4.4814
Epoch 3/10
2577/2577 ━━━━━━━━━━━━━━━━━━━━ 379s 147ms/step - accuracy: 0.3346 - loss: 3.7417 - val_accuracy: 0.4320 - val_loss: 4.1463
Epoch 4/10
2577/2577 ━━━━━━━━━━━━━━━━━━━━ 379s 147ms/step - accuracy: 0.4305 - loss: 3.0122 - val_accuracy: 0.5086 - val_loss: 3.8335
Epoch 5/10
2577/2577 ━━━━━━━━━━━━━━━━━━━━ 377s 146ms/step - accuracy: 0.5252 - loss: 2.4041 - val_accuracy: 0.5959 - val_loss: 3.4851
Epoch 6/10
2577/2577 ━━━━━━━━━━━━━━━━━━━━ 379s 147ms/step - accuracy: 0.6059 - loss: 1.9214 - val_accuracy: 0.6295 - val_loss: 3.2763
Epoch 7/10
2577/2577 ━━━━━━━━━━━━━━━━━━━━ 378s 147ms/step - accuracy: 0.6528 - loss: 1.6197 - val_accuracy: 0.6541 - val_loss: 3.1422
Epoch 8/10
2577/2577 ━━━━━━━━━━━━━━━━━━━━ 383s 149ms/step - ac

In [ ]:
# Required imports
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LayerNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MultiHeadAttention, Add
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Set seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Adjust TensorFlow settings for CPU optimization based on the i7-13700K capabilities
tf.config.threading.set_intra_op_parallelism_threads(16)  # Utilize all available cores
tf.config.threading.set_inter_op_parallelism_threads(8)   # Optimize parallel execution between operations

# Set paths to the CSV files (update as needed)
base_path = r'C:\Users\user\Desktop\cw\MBA\instacart-market-basket-analysis'
orders_path = f'{base_path}\\orders.csv'
order_products_prior_path = f'{base_path}\\order_products__prior.csv'
products_path = f'{base_path}\\products.csv'

# Load data from CSV files
orders = pd.read_csv(orders_path)
order_products_prior = pd.read_csv(order_products_prior_path)
products = pd.read_csv(products_path)

# Preprocessing function (modify based on dataset specifics)
def preprocess_data(orders, order_products, max_sequence_length):
    # Merge orders with prior products to get sequences per user
    merged = pd.merge(order_products, orders, on='order_id')
    # Group by user_id to create sequences of product_ids
    sequences = merged.groupby('user_id')['product_id'].apply(list).values

    # Ensure sequences are not empty, otherwise filter them out
    filtered_sequences = [seq for seq in sequences if len(seq) > 1]
    labels = [seq[-1] for seq in filtered_sequences]  # Take the last item as the label

    # Pad sequences to ensure they have the same length
    padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(filtered_sequences, maxlen=max_sequence_length, padding='post')
    return padded_sequences, np.array(labels)

# Set hyperparameters
vocab_size = products['product_id'].nunique() + 1  # Example value based on products dataset
embedding_dim = 100  # Adjusted for better representation
num_heads = 4  # Number of attention heads
ff_dim = 256  # Feed-forward layer dimension
dropout_rate = 0.1  # Dropout rate for regularization
max_sequence_length = 200  # Example value

# Preprocess the data
X, y = preprocess_data(orders, order_products_prior, max_sequence_length)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Positional encoding function
def positional_encoding(maxlen, d_model):
    pos = np.arange(maxlen)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    pos_encoding = pos * angle_rates
    pos_encoding[:, 0::2] = np.sin(pos_encoding[:, 0::2])
    pos_encoding[:, 1::2] = np.cos(pos_encoding[:, 1::2])
    return tf.convert_to_tensor(pos_encoding, dtype=tf.float32)

# Transformer Encoder Layer for SASRec
def transformer_encoder(inputs, num_heads, ff_dim, dropout_rate):
    # Multi-head self-attention
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(inputs, inputs)
    attention_output = Dropout(dropout_rate)(attention_output)
    attention_output = Add()([inputs, attention_output])
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)

    # Feed-forward layer
    ff_output = Dense(ff_dim, activation='relu')(attention_output)
    ff_output = Dense(inputs.shape[-1])(ff_output)
    ff_output = Dropout(dropout_rate)(ff_output)
    ff_output = Add()([attention_output, ff_output])
    return LayerNormalization(epsilon=1e-6)(ff_output)

# Function to build SASRec model
def build_sasrec(vocab_size, max_sequence_length, embedding_dim, num_heads, ff_dim, dropout_rate):
    inputs = Input(shape=(max_sequence_length,))
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, mask_zero=True)(inputs)

    # Add positional encoding
    pos_encoding = positional_encoding(max_sequence_length, embedding_dim)
    embeddings = embedding_layer + pos_encoding

    # Add transformer encoder layers (e.g., 2 layers for SASRec)
    transformer_output = transformer_encoder(embeddings, num_heads, ff_dim, dropout_rate)
    transformer_output = transformer_encoder(transformer_output, num_heads, ff_dim, dropout_rate)

    # Output layer for prediction
    output = Dense(vocab_size, activation='softmax')(transformer_output[:, -1, :])

    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Build the SASRec model
sasrec_model = build_sasrec(vocab_size, max_sequence_length, embedding_dim, num_heads, ff_dim, dropout_rate)

# Create data generators for training and testing
batch_size = 32  # Example value

def create_generator(X, y, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.shuffle(buffer_size=1024).batch(batch_size).repeat()
    return dataset

train_generator = create_generator(X_train, y_train, batch_size)
test_generator = create_generator(X_test, y_test, batch_size)

# Determine steps per epoch
train_steps = len(X_train) // batch_size
test_steps = len(X_test) // batch_size

# Training the SASRec model
sasrec_model.fit(train_generator,
                 steps_per_epoch=train_steps,
                 epochs=10,  # Adjust epochs as needed
                 validation_data=test_generator,
                 validation_steps=test_steps)

# Evaluation function for SASRec model
def evaluate_model_generator(model, generator, steps, k=5):
    precision, recall, hit_rate, mrr, count = 0, 0, 0, 0, 0
    for X_batch, y_batch in generator:
        predictions = model.predict(X_batch, verbose=0)
        top_k_preds = np.argsort(predictions, axis=-1)[:, -k:]
        for i in range(len(y_batch)):
            true_item = y_batch[i]
            rank = np.where(top_k_preds[i] == true_item)[0]
            if len(rank) > 0:
                rank = rank[0] + 1
                precision += 1
                recall += 1
                hit_rate += 1
                mrr += 1 / rank
        count += len(y_batch)
        if count >= steps * batch_size:
            break
    precision /= count
    recall /= count
    hit_rate /= count
    mrr /= count
    return precision, recall, hit_rate, mrr

# Evaluate SASRec model
sasrec_results = evaluate_model_generator(sasrec_model, test_generator, test_steps)
print("SASRec Performance (Precision@5, Recall@5, Hit Rate, MRR):", sasrec_results)
import csv
import os

# Save results to a CSV file
def save_results_to_csv(results, base_path, filename):
    filepath = os.path.join(base_path, filename)
    with open(filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Metric', 'Value'])
        writer.writerow(['Precision@5', results[0]])
        writer.writerow(['Recall@5', results[1]])
        writer.writerow(['Hit Rate', results[2]])
        writer.writerow(['MRR', results[3]])

# Example usage
save_results_to_csv(sasrec_results, base_path, 'sasrec_resultsO1.csv')


Epoch 1/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 1514s 293ms/step - accuracy: 0.0081 - loss: 9.3493 - val_accuracy: 0.0110 - val_loss: 8.6932
Epoch 2/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 1511s 293ms/step - accuracy: 0.0120 - loss: 8.1359 - val_accuracy: 0.0117 - val_loss: 8.7496
Epoch 3/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 1507s 292ms/step - accuracy: 0.0110 - loss: 7.9894 - val_accuracy: 0.0138 - val_loss: 8.8461
Epoch 4/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 1504s 292ms/step - accuracy: 0.0123 - loss: 7.8917 - val_accuracy: 0.0116 - val_loss: 8.9128
Epoch 5/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 1506s 292ms/step - accuracy: 0.0124 - loss: 7.8322 - val_accuracy: 0.0145 - val_loss: 8.9884
Epoch 6/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 1479s 287ms/step - accuracy: 0.0135 - loss: 7.7571 - val_accuracy: 0.0146 - val_loss: 8.9779
Epoch 7/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 1497s 290ms/step - accuracy: 0.0150 - loss: 7.7091 - val_accuracy: 0.0173 - val_loss: 9.0138
Epoch 8/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 1503s 292ms/s

In [ ]:
# Required imports
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LayerNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MultiHeadAttention, Add
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Set seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Adjust TensorFlow settings for CPU optimization based on the i7-13700K capabilities
tf.config.threading.set_intra_op_parallelism_threads(16)  # Utilize all available cores
tf.config.threading.set_inter_op_parallelism_threads(8)   # Optimize parallel execution between operations

# Set paths to the CSV files (update as needed)
base_path = r'C:\Users\user\Desktop\cw\MBA\instacart-market-basket-analysis'
orders_path = f'{base_path}\\orders.csv'
order_products_prior_path = f'{base_path}\\order_products__prior.csv'
products_path = f'{base_path}\\products.csv'

# Load data from CSV files
orders = pd.read_csv(orders_path)
order_products_prior = pd.read_csv(order_products_prior_path)
products = pd.read_csv(products_path)

# Preprocessing function (modify based on dataset specifics)
def preprocess_data(orders, order_products, max_sequence_length):
    # Merge orders with prior products to get sequences per user
    merged = pd.merge(order_products, orders, on='order_id')
    # Group by user_id to create sequences of product_ids
    sequences = merged.groupby('user_id')['product_id'].apply(list).values

    # Ensure sequences are not empty, otherwise filter them out
    filtered_sequences = [seq for seq in sequences if len(seq) > 1]
    labels = [seq[-1] for seq in filtered_sequences]  # Take the last item as the label

    # Pad sequences to ensure they have the same length
    padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(filtered_sequences, maxlen=max_sequence_length, padding='post')
    return padded_sequences, np.array(labels)

# Set hyperparameters
vocab_size = products['product_id'].nunique() + 1  # Example value based on products dataset
embedding_dim = 150  # Adjusted for better representation
num_heads = 4  # Number of attention heads
ff_dim = 256  # Feed-forward layer dimension
dropout_rate = 0.1  # Dropout rate for regularization
max_sequence_length = 100  # Example value

# Preprocess the data
X, y = preprocess_data(orders, order_products_prior, max_sequence_length)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Positional encoding function
def positional_encoding(maxlen, d_model):
    pos = np.arange(maxlen)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    pos_encoding = pos * angle_rates
    pos_encoding[:, 0::2] = np.sin(pos_encoding[:, 0::2])
    pos_encoding[:, 1::2] = np.cos(pos_encoding[:, 1::2])
    return tf.convert_to_tensor(pos_encoding, dtype=tf.float32)

# Transformer Encoder Layer for SASRec
def transformer_encoder(inputs, num_heads, ff_dim, dropout_rate):
    # Multi-head self-attention
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(inputs, inputs)
    attention_output = Dropout(dropout_rate)(attention_output)
    attention_output = Add()([inputs, attention_output])
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)

    # Feed-forward layer
    ff_output = Dense(ff_dim, activation='relu')(attention_output)
    ff_output = Dense(inputs.shape[-1])(ff_output)
    ff_output = Dropout(dropout_rate)(ff_output)
    ff_output = Add()([attention_output, ff_output])
    return LayerNormalization(epsilon=1e-6)(ff_output)

# Function to build SASRec model
def build_sasrec(vocab_size, max_sequence_length, embedding_dim, num_heads, ff_dim, dropout_rate):
    inputs = Input(shape=(max_sequence_length,))
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, mask_zero=True)(inputs)

    # Add positional encoding
    pos_encoding = positional_encoding(max_sequence_length, embedding_dim)
    embeddings = embedding_layer + pos_encoding

    # Add transformer encoder layers (e.g., 2 layers for SASRec)
    transformer_output = transformer_encoder(embeddings, num_heads, ff_dim, dropout_rate)
    transformer_output = transformer_encoder(transformer_output, num_heads, ff_dim, dropout_rate)

    # Output layer for prediction
    output = Dense(vocab_size, activation='softmax')(transformer_output[:, -1, :])

    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Build the SASRec model
sasrec_model = build_sasrec(vocab_size, max_sequence_length, embedding_dim, num_heads, ff_dim, dropout_rate)

# Create data generators for training and testing
batch_size = 32  # Example value

def create_generator(X, y, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.shuffle(buffer_size=1024).batch(batch_size).repeat()
    return dataset

train_generator = create_generator(X_train, y_train, batch_size)
test_generator = create_generator(X_test, y_test, batch_size)

# Determine steps per epoch
train_steps = len(X_train) // batch_size
test_steps = len(X_test) // batch_size

# Training the SASRec model
sasrec_model.fit(train_generator,
                 steps_per_epoch=train_steps,
                 epochs=10,  # Adjust epochs as needed
                 validation_data=test_generator,
                 validation_steps=test_steps)

# Evaluation function for SASRec model
def evaluate_model_generator(model, generator, steps, k=5):
    precision, recall, hit_rate, mrr, count = 0, 0, 0, 0, 0
    for X_batch, y_batch in generator:
        predictions = model.predict(X_batch, verbose=0)
        top_k_preds = np.argsort(predictions, axis=-1)[:, -k:]
        for i in range(len(y_batch)):
            true_item = y_batch[i]
            rank = np.where(top_k_preds[i] == true_item)[0]
            if len(rank) > 0:
                rank = rank[0] + 1
                precision += 1
                recall += 1
                hit_rate += 1
                mrr += 1 / rank
        count += len(y_batch)
        if count >= steps * batch_size:
            break
    precision /= count
    recall /= count
    hit_rate /= count
    mrr /= count
    return precision, recall, hit_rate, mrr

# Evaluate SASRec model
sasrec_results = evaluate_model_generator(sasrec_model, test_generator, test_steps)
print("SASRec Performance (Precision@5, Recall@5, Hit Rate, MRR):", sasrec_results)
import csv
import os

# Save results to a CSV file
def save_results_to_csv(results, base_path, filename):
    filepath = os.path.join(base_path, filename)
    with open(filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Metric', 'Value'])
        writer.writerow(['Precision@5', results[0]])
        writer.writerow(['Recall@5', results[1]])
        writer.writerow(['Hit Rate', results[2]])
        writer.writerow(['MRR', results[3]])

# Example usage
save_results_to_csv(sasrec_results, base_path, 'sasrec_resultsO2.csv')


Epoch 1/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 1379s 267ms/step - accuracy: 0.0077 - loss: 9.3780 - val_accuracy: 0.0123 - val_loss: 8.9107
Epoch 2/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 1332s 258ms/step - accuracy: 0.0131 - loss: 8.2052 - val_accuracy: 0.0141 - val_loss: 8.9171
Epoch 3/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 1309s 254ms/step - accuracy: 0.0128 - loss: 7.9949 - val_accuracy: 0.0152 - val_loss: 8.6960
Epoch 4/10
5155/5155 ━━━━━━━━━━━━━━━━━━━━ 1310s 254ms/step - accuracy: 0.0150 - loss: 7.7109 - val_accuracy: 0.0176 - val_loss: 8.7005
Epoch 5/10
5144/5155 ━━━━━━━━━━━━━━━━━━━━ 2s 231ms/step - accuracy: 0.0189 - loss: 7.4412

In [ ]:
# Required imports
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LayerNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MultiHeadAttention, Add
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Set seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Adjust TensorFlow settings for CPU optimization based on the i7-13700K capabilities
tf.config.threading.set_intra_op_parallelism_threads(16)  # Utilize all available cores
tf.config.threading.set_inter_op_parallelism_threads(8)   # Optimize parallel execution between operations

# Set paths to the CSV files (update as needed)
base_path = r'C:\Users\user\Desktop\cw\MBA\instacart-market-basket-analysis'
orders_path = f'{base_path}\\orders.csv'
order_products_prior_path = f'{base_path}\\order_products__prior.csv'
products_path = f'{base_path}\\products.csv'

# Load data from CSV files
orders = pd.read_csv(orders_path)
order_products_prior = pd.read_csv(order_products_prior_path)
products = pd.read_csv(products_path)

# Preprocessing function (modify based on dataset specifics)
def preprocess_data(orders, order_products, max_sequence_length):
    # Merge orders with prior products to get sequences per user
    merged = pd.merge(order_products, orders, on='order_id')
    # Group by user_id to create sequences of product_ids
    sequences = merged.groupby('user_id')['product_id'].apply(list).values

    # Ensure sequences are not empty, otherwise filter them out
    filtered_sequences = [seq for seq in sequences if len(seq) > 1]
    labels = [seq[-1] for seq in filtered_sequences]  # Take the last item as the label

    # Pad sequences to ensure they have the same length
    padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(filtered_sequences, maxlen=max_sequence_length, padding='post')
    return padded_sequences, np.array(labels)

# Set hyperparameters
vocab_size = products['product_id'].nunique() + 1  # Example value based on products dataset
embedding_dim = 100  # Adjusted for better representation
num_heads = 2 # Number of attention heads
ff_dim = 256  # Feed-forward layer dimension
dropout_rate = 0.2  # Dropout rate for regularization
max_sequence_length = 50  # Example value

# Preprocess the data
X, y = preprocess_data(orders, order_products_prior, max_sequence_length)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Positional encoding function
def positional_encoding(maxlen, d_model):
    pos = np.arange(maxlen)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    pos_encoding = pos * angle_rates
    pos_encoding[:, 0::2] = np.sin(pos_encoding[:, 0::2])
    pos_encoding[:, 1::2] = np.cos(pos_encoding[:, 1::2])
    return tf.convert_to_tensor(pos_encoding, dtype=tf.float32)

# Transformer Encoder Layer for SASRec
def transformer_encoder(inputs, num_heads, ff_dim, dropout_rate):
    # Multi-head self-attention
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(inputs, inputs)
    attention_output = Dropout(dropout_rate)(attention_output)
    attention_output = Add()([inputs, attention_output])
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)

    # Feed-forward layer
    ff_output = Dense(ff_dim, activation='relu')(attention_output)
    ff_output = Dense(inputs.shape[-1])(ff_output)
    ff_output = Dropout(dropout_rate)(ff_output)
    ff_output = Add()([attention_output, ff_output])
    return LayerNormalization(epsilon=1e-6)(ff_output)

# Function to build SASRec model
def build_sasrec(vocab_size, max_sequence_length, embedding_dim, num_heads, ff_dim, dropout_rate):
    inputs = Input(shape=(max_sequence_length,))
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, mask_zero=True)(inputs)

    # Add positional encoding
    pos_encoding = positional_encoding(max_sequence_length, embedding_dim)
    embeddings = embedding_layer + pos_encoding

    # Add transformer encoder layers (e.g., 2 layers for SASRec)
    transformer_output = transformer_encoder(embeddings, num_heads, ff_dim, dropout_rate)
    transformer_output = transformer_encoder(transformer_output, num_heads, ff_dim, dropout_rate)

    # Output layer for prediction
    output = Dense(vocab_size, activation='softmax')(transformer_output[:, -1, :])

    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Build the SASRec model
sasrec_model = build_sasrec(vocab_size, max_sequence_length, embedding_dim, num_heads, ff_dim, dropout_rate)

# Create data generators for training and testing
batch_size = 32  # Example value

def create_generator(X, y, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.shuffle(buffer_size=1024).batch(batch_size).repeat()
    return dataset

train_generator = create_generator(X_train, y_train, batch_size)
test_generator = create_generator(X_test, y_test, batch_size)

# Determine steps per epoch
train_steps = len(X_train) // batch_size
test_steps = len(X_test) // batch_size

# Training the SASRec model
sasrec_model.fit(train_generator,
                 steps_per_epoch=train_steps,
                 epochs=10,  # Adjust epochs as needed
                 validation_data=test_generator,
                 validation_steps=test_steps)

# Evaluation function for SASRec model
def evaluate_model_generator(model, generator, steps, k=5):
    precision, recall, hit_rate, mrr, count = 0, 0, 0, 0, 0
    for X_batch, y_batch in generator:
        predictions = model.predict(X_batch, verbose=0)
        top_k_preds = np.argsort(predictions, axis=-1)[:, -k:]
        for i in range(len(y_batch)):
            true_item = y_batch[i]
            rank = np.where(top_k_preds[i] == true_item)[0]
            if len(rank) > 0:
                rank = rank[0] + 1
                precision += 1
                recall += 1
                hit_rate += 1
                mrr += 1 / rank
        count += len(y_batch)
        if count >= steps * batch_size:
            break
    precision /= count
    recall /= count
    hit_rate /= count
    mrr /= count
    return precision, recall, hit_rate, mrr

# Evaluate SASRec model
sasrec_results = evaluate_model_generator(sasrec_model, test_generator, test_steps)
print("SASRec Performance (Precision@5, Recall@5, Hit Rate, MRR):", sasrec_results)
import csv
import os

# Save results to a CSV file
def save_results_to_csv(results, base_path, filename):
    filepath = os.path.join(base_path, filename)
    with open(filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Metric', 'Value'])
        writer.writerow(['Precision@5', results[0]])
        writer.writerow(['Recall@5', results[1]])
        writer.writerow(['Hit Rate', results[2]])
        writer.writerow(['MRR', results[3]])

# Example usage
save_results_to_csv(sasrec_results, base_path, 'sasrec_resultsO3.csv')


In [ ]:
# Required imports
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Embedding, LayerNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import MultiHeadAttention, Add
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# Set seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Adjust TensorFlow settings for CPU optimization based on the i7-13700K capabilities
tf.config.threading.set_intra_op_parallelism_threads(16)  # Utilize all available cores
tf.config.threading.set_inter_op_parallelism_threads(8)   # Optimize parallel execution between operations

# Set paths to the CSV files (update as needed)
base_path = r'C:\Users\user\Desktop\cw\MBA\instacart-market-basket-analysis'
orders_path = f'{base_path}\\orders.csv'
order_products_prior_path = f'{base_path}\\order_products__prior.csv'
products_path = f'{base_path}\\products.csv'

# Load data from CSV files
orders = pd.read_csv(orders_path)
order_products_prior = pd.read_csv(order_products_prior_path)
products = pd.read_csv(products_path)

# Preprocessing function (modify based on dataset specifics)
def preprocess_data(orders, order_products, max_sequence_length):
    # Merge orders with prior products to get sequences per user
    merged = pd.merge(order_products, orders, on='order_id')
    # Group by user_id to create sequences of product_ids
    sequences = merged.groupby('user_id')['product_id'].apply(list).values

    # Ensure sequences are not empty, otherwise filter them out
    filtered_sequences = [seq for seq in sequences if len(seq) > 1]
    labels = [seq[-1] for seq in filtered_sequences]  # Take the last item as the label

    # Pad sequences to ensure they have the same length
    padded_sequences = tf.keras.preprocessing.sequence.pad_sequences(filtered_sequences, maxlen=max_sequence_length, padding='post')
    return padded_sequences, np.array(labels)

# Set hyperparameters
vocab_size = products['product_id'].nunique() + 1  # Example value based on products dataset
embedding_dim = 100  # Adjusted for better representation
num_heads = 4  # Number of attention heads
ff_dim = 256  # Feed-forward layer dimension
dropout_rate = 0.2  # Dropout rate for regularization
max_sequence_length = 50  # Example value

# Preprocess the data
X, y = preprocess_data(orders, order_products_prior, max_sequence_length)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Positional encoding function
def positional_encoding(maxlen, d_model):
    pos = np.arange(maxlen)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    pos_encoding = pos * angle_rates
    pos_encoding[:, 0::2] = np.sin(pos_encoding[:, 0::2])
    pos_encoding[:, 1::2] = np.cos(pos_encoding[:, 1::2])
    return tf.convert_to_tensor(pos_encoding, dtype=tf.float32)

# Transformer Encoder Layer for SASRec
def transformer_encoder(inputs, num_heads, ff_dim, dropout_rate):
    # Multi-head self-attention
    attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=inputs.shape[-1])(inputs, inputs)
    attention_output = Dropout(dropout_rate)(attention_output)
    attention_output = Add()([inputs, attention_output])
    attention_output = LayerNormalization(epsilon=1e-6)(attention_output)

    # Feed-forward layer
    ff_output = Dense(ff_dim, activation='relu')(attention_output)
    ff_output = Dense(inputs.shape[-1])(ff_output)
    ff_output = Dropout(dropout_rate)(ff_output)
    ff_output = Add()([attention_output, ff_output])
    return LayerNormalization(epsilon=1e-6)(ff_output)

# Function to build SASRec model
def build_sasrec(vocab_size, max_sequence_length, embedding_dim, num_heads, ff_dim, dropout_rate):
    inputs = Input(shape=(max_sequence_length,))
    embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim, mask_zero=True)(inputs)

    # Add positional encoding
    pos_encoding = positional_encoding(max_sequence_length, embedding_dim)
    embeddings = embedding_layer + pos_encoding

    # Add transformer encoder layers (e.g., 2 layers for SASRec)
    transformer_output = transformer_encoder(embeddings, num_heads, ff_dim, dropout_rate)
    transformer_output = transformer_encoder(transformer_output, num_heads, ff_dim, dropout_rate)

    # Output layer for prediction
    output = Dense(vocab_size, activation='softmax')(transformer_output[:, -1, :])

    model = Model(inputs=inputs, outputs=output)
    model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Build the SASRec model
sasrec_model = build_sasrec(vocab_size, max_sequence_length, embedding_dim, num_heads, ff_dim, dropout_rate)

# Create data generators for training and testing
batch_size = 32  # Example value

def create_generator(X, y, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    dataset = dataset.shuffle(buffer_size=1024).batch(batch_size).repeat()
    return dataset

train_generator = create_generator(X_train, y_train, batch_size)
test_generator = create_generator(X_test, y_test, batch_size)

# Determine steps per epoch
train_steps = len(X_train) // batch_size
test_steps = len(X_test) // batch_size

# Training the SASRec model
sasrec_model.fit(train_generator,
                 steps_per_epoch=train_steps,
                 epochs=10,  # Adjust epochs as needed
                 validation_data=test_generator,
                 validation_steps=test_steps)

# Evaluation function for SASRec model
def evaluate_model_generator(model, generator, steps, k=5):
    precision, recall, hit_rate, mrr, count = 0, 0, 0, 0, 0
    for X_batch, y_batch in generator:
        predictions = model.predict(X_batch, verbose=0)
        top_k_preds = np.argsort(predictions, axis=-1)[:, -k:]
        for i in range(len(y_batch)):
            true_item = y_batch[i]
            rank = np.where(top_k_preds[i] == true_item)[0]
            if len(rank) > 0:
                rank = rank[0] + 1
                precision += 1
                recall += 1
                hit_rate += 1
                mrr += 1 / rank
        count += len(y_batch)
        if count >= steps * batch_size:
            break
    precision /= count
    recall /= count
    hit_rate /= count
    mrr /= count
    return precision, recall, hit_rate, mrr

# Evaluate SASRec model
sasrec_results = evaluate_model_generator(sasrec_model, test_generator, test_steps)
print("SASRec Performance (Precision@5, Recall@5, Hit Rate, MRR):", sasrec_results)
import csv
import os

# Save results to a CSV file
def save_results_to_csv(results, base_path, filename):
    filepath = os.path.join(base_path, filename)
    with open(filepath, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Metric', 'Value'])
        writer.writerow(['Precision@5', results[0]])
        writer.writerow(['Recall@5', results[1]])
        writer.writerow(['Hit Rate', results[2]])
        writer.writerow(['MRR', results[3]])

# Example usage
save_results_to_csv(sasrec_results, base_path, 'sasrec_resultsO4.csv')
